# Download COMBAT data

In [2]:
%%bash 

mkdir data
wget -P data https://zenodo.org/record/6120249/files/COMBAT-CITESeq-DATA.h5ad

mkdir: data: File exists
--2023-05-09 13:59:42--  https://zenodo.org/record/6120249/files/COMBAT-CITESeq-DATA.h5ad
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6409089483 (6,0G) [application/octet-stream]
Saving to: ‘data/COMBAT-CITESeq-DATA.h5ad’

     0K .......... .......... .......... .......... ..........  0% 2,31M 44m12s
    50K .......... .......... .......... .......... ..........  0% 1,15M 66m32s
   100K .......... .......... .......... .......... ..........  0% 1,09M 75m27s
   150K .......... .......... .......... .......... ..........  0%  663K 95m55s
   200K .......... .......... .......... .......... ..........  0%  870K 1h40m
   250K .......... .......... .......... .......... ..........  0%  394K 2h8m
   300K .......... .......... .......... .......... ..........  0% 1,14M 2h2m
   350K .......... .......... .......... .......... ..........

  5000K .......... .......... .......... .......... ..........  0%  746K 2h4m
  5050K .......... .......... .......... .......... ..........  0% 1,28M 2h4m
  5100K .......... .......... .......... .......... ..........  0% 2,24M 2h3m
  5150K .......... .......... .......... .......... ..........  0%  241K 2h6m
  5200K .......... .......... .......... .......... ..........  0% 1,06M 2h6m
  5250K .......... .......... .......... .......... ..........  0%  986K 2h6m
  5300K .......... .......... .......... .......... ..........  0%  735K 2h6m
  5350K .......... .......... .......... .......... ..........  0%  281K 2h8m
  5400K .......... .......... .......... .......... ..........  0% 1,76M 2h7m
  5450K .......... .......... .......... .......... ..........  0% 1,07M 2h7m
  5500K .......... .......... .......... .......... ..........  0% 1,45M 2h7m
  5550K .......... .......... .......... .......... ..........  0% 1,61M 2h6m
  5600K .......... .......... .......... .......... ..........  

Process is interrupted.


# Read data

In [25]:
import scanpy as sc
import scvi

Global seed set to 0
/Users/vladimir.shitov/miniconda3/envs/patients_representation/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/vladimir.shitov/miniconda3/envs/patients_representation/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [8]:
# ADATA_PATH = "data/COMBAT-CITESeq-DATA.h5ad"
ADATA_PATH = "/Users/vladimir.shitov/Documents/science/PhD/2022_10_patients_representation/data/COMBAT data/COMBAT-CITESeq-DATA.h5ad"

In [9]:
adata = sc.read_h5ad(ADATA_PATH)
adata

AnnData object with n_obs × n_vars = 836148 × 20807
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_H

COMBAT data is multimodal i.e. it contains protein expression as well. We will leave it behind for now and focus only on the RNA expression data

In [10]:
adata = adata[:, adata.var["feature_types"] == "Gene Expression"].copy()

Find highly variable genes

In [11]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, flavor="seurat_v3",
                            n_top_genes=3000)

/Users/vladimir.shitov/miniconda3/envs/patients_representation/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


In [12]:
adata = adata[:, adata.var.highly_variable].copy()

In [14]:
adata.var

,gene_ids,feature_types,highly_variable,highly_variable_rank,means,variances,variances_norm
ISG15,ENSG00000187608,Gene Expression,True,723.0,0.565940,0.784620,1.320167
TNFRSF18,ENSG00000186891,Gene Expression,True,2496.0,0.032802,0.048503,1.091563
MMP23B,ENSG00000189409,Gene Expression,True,2316.0,0.010488,0.015635,1.100135
CFAP74,ENSG00000142609,Gene Expression,True,2030.0,0.000077,0.000113,1.117553
PLCH2,ENSG00000149527,Gene Expression,True,2786.0,0.005693,0.008328,1.079405
...,...,...,...,...,...,...,...
AC136616.2,ENSG00000277761,Gene Expression,True,1740.0,0.000145,0.000218,1.139804
AC141272.1,ENSG00000277836,Gene Expression,True,552.0,0.000406,0.000755,1.395083
AC007325.4,ENSG00000278817,Gene Expression,True,1604.0,0.000250,0.000381,1.153238
AC233755.2,ENSG00000277856,Gene Expression,True,202.0,0.023617,0.053809,1.676403


In [13]:
adata

AnnData object with n_obs × n_vars = 836148 × 3000
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC

In [16]:
sc.tl.pca(adata)

In [15]:
adata

AnnData object with n_obs × n_vars = 836148 × 3000
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC

In [23]:
adata.layers["raw"][:20, :20].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.,
        0., 0., 0., 0.],
       [8., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 2., 3., 1., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 2., 1., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [28]:
adata

AnnData object with n_obs × n_vars = 836148 × 3000
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC

Obtain scVI embedding

In [38]:
adata.raw = adata
scvi.model.SCVI.setup_anndata(adata, layer="raw", batch_key="scRNASeq_sample_ID")
vae = scvi.model.SCVI(adata, n_layers=2, n_latent=30, gene_likelihood="nb")
vae.train()
adata.obsm["X_scVI"] = vae.get_latent_representation()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/vladimir.shitov/miniconda3/envs/patients_representation/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


Epoch 10/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [28:32<00:00, 173.29s/it, loss=578, v_num=1]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [28:32<00:00, 171.30s/it, loss=578, v_num=1]


Obtain scANVI embedding

In [43]:
adata.obs["Annotation_major_subset"]

AAACCTGAGAAAGTGG-1-gPlexA1        NK
AAACCTGAGCGGATCA-1-gPlexA1       CD8
AAACCTGAGGACATTA-1-gPlexA1       nan
AAACCTGAGGCGACAT-1-gPlexA1    ncMono
AAACCTGAGGGAACGG-1-gPlexA1     cMono
                               ...  
TTTGTCAGTGGCAAAC-1-gPlexK7     cMono
TTTGTCAGTTACCGAT-1-gPlexK7       CD8
TTTGTCATCCTCTAGC-1-gPlexK7       CD8
TTTGTCATCGAGGTAG-1-gPlexK7     cMono
TTTGTCATCTCCCTGA-1-gPlexK7     cMono
Name: Annotation_major_subset, Length: 836148, dtype: category
Categories (18, object): ['B', 'CD4', 'CD8', 'DC', ..., 'cMono', 'iNKT', 'nan', 'ncMono']

In [41]:
adata.obs["Annotation_major_subset"].value_counts()

CD4       269661
cMono     213697
CD8       106025
NK         69927
nan        52444
B          44256
ncMono     33172
GDT         9580
DC          8959
PB          8596
DP          6561
DN          4557
MAIT        4541
HSC         1868
PLT         1563
iNKT         372
RET          304
Mast          65
Name: Annotation_major_subset, dtype: int64

In [ ]:
lvae = scvi.model.SCANVI.from_scvi_model(
    vae,
    adata=adata,
    labels_key="Annotation_major_subset",
    unlabeled_category="nan",
)
lvae.train(max_epochs=20, n_samples_per_label=100)
adata.obsm["X_scANVI"] = lvae.get_latent_representation(adata)

INFO     Training for 20 epochs.                                                                                   


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/vladimir.shitov/miniconda3/envs/patients_representation/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


Epoch 4/20:  15%|██████████████████████████▎                                                                                                                                                    | 3/20 [13:37<1:16:59, 271.75s/it, loss=667, v_num=1]